In [3]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import imghdr
import pandas as pd

C:\Users\biswa\AppData\Local\Temp\ipykernel_14928\3925996245.py:6: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


Load Data

In [4]:
data_dir = 'Affectnet Dataset'
data=pd.read_csv('Affectnet Dataset\labels.csv')

<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\biswa\AppData\Local\Temp\ipykernel_14928\3271766593.py:2: SyntaxWarning: invalid escape sequence '\l'
  data=pd.read_csv('Affectnet Dataset\labels.csv')


In [5]:
data.head()

,Unnamed: 0,pth,label,relFCs
0,0,anger/image0000006.jpg,surprise,0.873142
1,1,anger/image0000060.jpg,anger,0.852311
2,2,anger/image0000061.jpg,anger,0.800957
3,3,anger/image0000066.jpg,disgust,0.843079
4,4,anger/image0000106.jpg,anger,0.849108


In [ ]:
data.info()

In [6]:
X = data['pth'].values
y = data['label'].values

In [7]:
X

array(['anger/image0000006.jpg', 'anger/image0000060.jpg',
       'anger/image0000061.jpg', ..., 'surprise/image0034966.jpg',
       'surprise/image0034973.jpg', 'surprise/image0042075.jpg'],
      dtype=object)

In [ ]:
'''img = cv2.imread(data_dir+"/"+X[4]) 
#img = cv2.imread(data_dir+"/"+image_path) 
clf = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    #faces contains coordinates of the face
faces = clf.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
print(f'Faces : {faces}\n')
    
input_image = cv2.cvtColor(img[faces[0][1]:(faces[0][1]+faces[0][3]),faces[0][0]:(faces[0][0]+faces[0][3])],cv2.COLOR_BGR2RGB)   '''
    
#input_image = cv2.cvtColor(img[faces[0][1]:(faces[0][1]+faces[0][3]),faces[0][0]:(faces[0][0]+faces[0][3])],cv2.COLOR_BGR2RGB)   

In [9]:
print(X.size)
print()

28175


In [11]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import cv2

def preprocess_image(image_path):
    img = cv2.imread(data_dir+"/"+image_path) 
    clf = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    #faces contains coordinates of the face
    faces = clf.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    #print(f'Faces : {faces}\n')
    
    input_image = cv2.cvtColor(img[faces[0][1]:(faces[0][1]+faces[0][3]),faces[0][0]:(faces[0][0]+faces[0][3])],cv2.COLOR_BGR2RGB)   
    img_normalized = input_image / 255.0
    return img_normalized



In [20]:
X_preprocessed=[]
count=0
for image_path in X:
    count+=1
    print(count)
    print(image_path)
    X_preprocessed.append(preprocess_image(image_path))
#X_preprocessed=np.array([preprocess_image(image_path) for image_path in X])
X_preprocessed=np.array(X_preprocessed)

le = LabelEncoder()
y_preprocessed = to_categorical(le.fit_transform(y))

1
anger/image0000006.jpg
2
anger/image0000060.jpg
3
anger/image0000061.jpg
4
anger/image0000066.jpg
5
anger/image0000106.jpg
6
anger/image0000132.jpg
7
anger/image0000138.jpg
8
anger/image0000182.jpg
9
anger/image0000195.jpg
10
anger/image0000213.jpg
11
anger/image0000228.jpg
12
anger/image0000294.jpg
13
anger/image0000333.jpg
14
anger/image0000343.jpg
15
anger/image0000346.jpg
16
anger/image0000356.jpg
17
anger/image0000368.jpg
18
anger/image0000374.jpg
19
anger/image0000390.jpg
20
anger/image0000399.jpg
21
anger/image0000407.jpg


IndexError: tuple index out of range

In [ ]:
#X_preprocessed=np.array([preprocess_image(image_path) for image_path in X])
# data_dir='Affectnet Dataset'
# X_preprocessed=[]

# for image_path in X:
#     img = cv2.imread(data_dir+"/"+image_path)    
#     img_normalized = img/255.0
#     X_preprocessed.append(img_normalized)
#     # 


# X_preprocessed=np.array(X_preprocessed)



Spliting Test train 


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_preprocessed, test_size=0.2, random_state=42, stratify=y)

Creating CNN model

Building Deep learning model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3,3), 1, activation='relu', input_shape=(96,96,3)))#Convolution layer 1
model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))#Pooling layer 1
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), 1, activation='relu'))#Convolution layer 2
model.add(MaxPooling2D(pool_size=(2,2)))#Pooling layer 2
model.add(Conv2D(16, (3,3), 1, activation='relu'))#Convolution layer 3
model.add(MaxPooling2D(pool_size=(2,2)))#Pooling layer 3
model.add(Dropout(0.25))

model.add(Flatten())#Fully Connected layer
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))


#model.add(Dense(256, activation='relu'))
model.add(Dense(8, activation='softmax'))#Output layer

In [ ]:
from keras import metrics

# model.compile(loss='mean_squared_error', optimizer='sgd',
#               metrics=[metrics.mae,
#                        metrics.categorical_accuracy])

model.compile(optimizer='adam', 
          loss='categorical_crossentropy', 
          metrics=['accuracy'])

model.summary()

In [ ]:
#model.compile('sgd', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

Visualize CNN model

In [ ]:
import visualkeras
visualkeras.layered_view(model).show() # display using your system viewer
visualkeras.layered_view(model, to_file='Neural_Network_Visualize.png') # write to disk
visualkeras.layered_view(model, to_file='Neural_Network_Visualize.png').show() # write and show

Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
Log_history = model.fit(X_train,y_train, epochs=25, batch_size=64,validation_split=0.2,callbacks=[tensorboard_callback])

In [ ]:
model.save_weights('models/emotion_latest.h5')

In [ ]:
import pandas as pd
pd.DataFrame(Log_history.history).plot()

Performance

In [ ]:
label_mapping={0: 'anger',
 1: 'contempt',
 2: 'disgust',
 3: 'fear',
 4: 'happy',
 5: 'neutral',
 6: 'sad',
 7: 'surprise'}

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_score, recall_score, f1_score

y_pred_prob=model.predict(X_test)
# Convert probabilities to binary predictions
y_pred_labels = np.argmax(y_pred_prob, axis=1)

# Convert y_test to binary labels
y_true_labels = np.argmax(y_test, axis=1)

# List to store individual precision, recall, and F1-score values for each class
class_metrics = []

# List to store individual AUC values and ROC curves for each class
roc_curves = []


for i in range(8):
    # Convert predicted and true labels to binary labels
    y_pred_binary = (y_pred_labels == i)
    y_true_binary = (y_true_labels == i)
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(y_true_binary, y_pred_binary)
    recall = recall_score(y_true_binary, y_pred_binary)
    f1 = f1_score(y_true_binary, y_pred_binary)
    
    # ROC curve calculation
    fpr, tpr, _ = roc_curve(y_true_binary, y_pred_prob[:, i])
    roc_auc = auc(fpr, tpr)
    roc_curves.append((fpr, tpr, label_mapping[i]))
    
    # Store metrics in a dictionary
    class_metrics.append({
        'Label': label_mapping[i],
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'AUC': roc_auc
    })


    
# Compute and plot macro-average ROC curve
mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.mean([np.interp(mean_fpr, fpr, tpr) for fpr, tpr, _ in roc_curves], axis=0)
macro_auc = auc(mean_fpr, mean_tpr)




# Calculate overall precision, recall, and F1-score
overall_precision = precision_score(y_true_labels, y_pred_labels, average='macro')
overall_recall = recall_score(y_true_labels, y_pred_labels, average='macro')
overall_f1 = f1_score(y_true_labels, y_pred_labels, average='macro')

class_metrics.append({
    'Label': 'Overall',
    'Precision': overall_precision,
    'Recall': overall_recall,
    'F1-score': overall_f1,
    'AUC': macro_auc
})


# Create a Pandas DataFrame from the list of metrics
metrics_df = pd.DataFrame(class_metrics)

# Optionally, print the metrics table
print(metrics_df)


# Plot ROC curve for each class
plt.figure(figsize=(12, 8))
lw = 2

    
for fpr, tpr, label in roc_curves:
    plt.plot(fpr, tpr, lw=lw, label='ROC curve (area = {:.2f}) for Emotion: {}'.format(roc_auc, label))

plt.plot(mean_fpr, mean_tpr, color='darkorange', linestyle='--', linewidth=2, label='Macro-average ROC curve (area = {:.2f})'.format(macro_auc))


plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Each Class')
plt.legend(loc='lower right')
plt.show()

Evaluate performance

Test

In [ ]:
img = cv2.imread('test/happy.jpg')
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
img.shape

In [ ]:
resize = tf.image.resize(img, (96,96))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
import numpy as np

In [ ]:
resize.shape

In [ ]:
np.expand_dims(resize,0).shape

In [ ]:

yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:

yhat

In [ ]:
yhat

In [ ]:
classify_result = np.argmax(yhat)
classify_result

In [ ]:
label_mapping={0: 'anger',
 1: 'contempt',
 2: 'disgust',
 3: 'fear',
 4: 'happy',
 5: 'neutral',
 6: 'sad',
 7: 'surprise'}

Save the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','Final_CNN_with25epochs.h5'))

In [ ]:
new_model = load_model(os.path.join('models','Final_CNN_with25epochs.h5'))

In [ ]:
new_model.predict(np.expand_dims(resize/255,0))

In [ ]:
plt.imshow(resize)